In [1]:
import torch
import torch.autograd
from torch.autograd import Variable

def mlp(x, Weight, Bias):
    f = Variable(x, requires_grad=False) # Initializing f with x

    # f = sigmoid(w.dot(x) + b)
    for i in range(len(Weight)):
        #print("f (",f.shape,"):\n",f,"\n")
        #print("Weight[",i,"] (",Weight[i].shape,"):\n",Weight[i],"\n")
        #print("Bias[",i,"] (",Bias[i].shape,"):\n",Bias[i],"\n")
        #print("Multiplication of matrices Weight[",i,"] & f (",torch.matmul(Weight[i], f).shape,"):\n",torch.matmul(Weight[i], f),"\n")
        #print("Sum of above & Bias[",i,"] (",(torch.matmul(Weight[i], f) + Bias[i]).shape,"):\n",torch.matmul(Weight[i], f) + Bias[i],"\n")
        #print("\n")
        f = torch.sigmoid(torch.matmul(Weight[i], f) + Bias[i])
    return f

## Part 1

In [2]:
sizes = [1,2,1]
N = 1 # input dimension

x = 10*torch.rand(sizes[0], N).double()

Weight = []
Bias = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)

f = mlp(x, Weight, Bias)

print(f.data)

tensor([[0.7065]], dtype=torch.float64)


## Part 2

In [3]:
sizes = [4, 2, 1]
N = 1 # input dimension

x = 10*torch.rand(sizes[0], N).double()

Weight = []
Bias = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)

f = mlp(x, Weight, Bias)

print(f.data)

tensor([[0.4441]], dtype=torch.float64)


### İris Dataset

In [4]:
import pandas as pd
import numpy as np

def logsumexp(b):
    return torch.log(1 + torch.exp(b))

# 3 layers: 4 inputs-2 hidden variables-1 output
sizes = [4, 2, 1]
N = 1 # input dimension

df_iris = pd.read_csv(u'data/iris.txt',sep=' ')

# Get İris data targets, set values bigger than 1 to 0
y = torch.from_numpy(np.array(df_iris['c'])).double()
y[y>1]=0
y.reshape(len(y), N)

# Get İris data features
x = torch.from_numpy(np.array(df_iris[['sl','sw','pl','pw']])).double()

Weight = []
Bias = []

# Initializing Weight and Bias with random values
for i in range(len(sizes)-1):
    # For layer i, Weights are a sizes[i+1] x sizes[i] matrix
    W = Variable(20*torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    
    # For layer i, Biases are a sizes[i+1] x 1 vector
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)

# Euclidian cost error function
Error = torch.nn.MSELoss(size_average=True)
    
# Iteration variables
eta = 0.005
MAX_ITER = 1000
EE = []

for epoch in range(MAX_ITER):  
    for i in range(len(y)):
        f = mlp(x[i,:].reshape(sizes[0], N), Weight, Bias) # converting [4] sized x vector to [4,1] matrix

        # Measure the error
        E = Error(f, y[i].reshape(sizes[-1], N)) # converting [1] sized y vector to [1,1] matrix
        EE.append(E.data.numpy())

        # Compute the derivative of the error with respect to Weights and Biases
        E.backward() 

        # Take the step and reset weights
        for j in range(len(sizes)-1):
            Weight[j].data.add_(-eta*Weight[j].grad.data)
            Bias[j].data.add_(-eta*Bias[j].grad.data)
            Weight[j].grad.zero_()
            Bias[j].grad.zero_()
    
    if(epoch%100==0):
        print("In epoch ", epoch, ", EE is:", EE[epoch])

result = torch.zeros(len(y))
        
for i in range(len(y)):
    result[i] = torch.round(mlp(x[i,:].reshape(sizes[0], N), Weight, Bias))

confusion_matrix = torch.zeros((2,2))

for i in range(len(result)):
    confusion_matrix[int(y[i] - 1), int(result[i] - 1)] += 1
    
print("Confusion matrix:\n", confusion_matrix, "\n")
    
accuracy = torch.sum(torch.diag(confusion_matrix))/torch.sum(confusion_matrix)

print("Accuracy:", accuracy*100, "%\n")

print("Final Error is:", EE[-1])

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In epoch  0 , EE is: 0.9995543394991954
In epoch  100 , EE is: 4.968651137121534e-08
In epoch  200 , EE is: 4.970864968617287e-08
In epoch  300 , EE is: 0.9995541410193017
In epoch  400 , EE is: 4.973079290883436e-08
In epoch  500 , EE is: 4.975296081996485e-08
In epoch  600 , EE is: 0.9995539423626378
In epoch  700 , EE is: 4.977513364753527e-08
In epoch  800 , EE is: 4.979733120760128e-08
In epoch  900 , EE is: 0.9995537435289678
Confusion matrix:
 tensor([[  0.,  50.],
        [  0., 100.]]) 

Accuracy: tensor(66.6667) %

Final Error is: 8.217252768461607e-08
